# Mistral and Weights & Biases

- Weights & Biases: https://wandb.ai/
- Mistral finetuning docs: https://docs.mistral.ai/capabilities/finetuning/
- Tracing with W&B Weave: https://wandb.me/weave

In [1]:
# !pip install mistralai pandas weave

## Using Mistral and Weave

You will probably integrate MistralAI API calls in your codebase by creating a function like the one below:

In [2]:
import os, asyncio
import weave
from mistralai.async_client import MistralAsyncClient
from mistralai.models.chat_completion import ChatMessage

client = MistralAsyncClient(api_key=os.environ["MISTRAL_API_KEY"])

@weave.op()  # <---- add this and you are good to go
async def call_mistral(model:str, messages:list, **kwargs) -> str:
    "Call the Mistral API"
    chat_response = await client.chat(
        model=model,
        messages=messages,
        **kwargs,
    )
    return chat_response.choices[0].message.content

The only thing you need to do is add the @weave.op() decorator to the function you want to trace.

Let's define a more interesting function that recommends cheese based on the region and model.



In [3]:
@weave.op()
async def cheese_recommender(region:str, model:str) -> str:
    "Recommend the best cheese in a given region"
     
    messages = [ChatMessage(
        role="user", 
        content=f"What is the best cheese in {region}?")]

    cheeses = await call_mistral(model=model, messages=messages)
    return {"region": region, "cheeses": cheeses}

Let's run this function and see how weave traces it. We call weave.init() to tell weave the project where to store the traces.

In [4]:

weave.init("mistral_webinar")
out = await cheese_recommender(region="France", model="open-mistral-7b")
print(out)

Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/mistral_webinar/weave
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/e30abb50-3879-43d7-8ae1-afb22d7f2bb1
{'region': 'France', 'cheeses': 'France is renowned for its diverse and high-quality cheeses, and the "best" cheese can often be subjective as it depends on personal taste. However, one of the most famous French cheeses is Roquefort, a blue cheese made from sheep\'s milk. It\'s known for its strong flavor and distinctive veining.\n\nAnother popular French cheese is Brie de Meaux, a soft, creamy cheese with a mild, slighty nutty flavor. Camembert, a relative of Brie, is also well-loved, especially outside of France.\n\nFor those who prefer harder cheeses, Comté, a nutty and slightly sweet cheese, is a good choice. And let\'s not forget about Munster, a soft, pungent cheese with a distinctive texture and strong aroma.\n\nEach region in France has its own unique cheeses, so there\'s a wide v

You can view the traces by clicking the link above 👆
![](cheese_recomender.png)



## Prepare the dataset

Some data from wandbot

In [5]:
import pandas as pd
df = pd.read_json('qa.jsonl', orient='records', lines=True)

In [6]:
df.head()

,question,answer
0,What is the difference between team and organi...,A team is a collaborative workspace for a grou...
1,What is the difference between team and entity...,A team is a collaborative workspace for a grou...
2,What is a team and where can I find more infor...,Use W&B Teams as a central workspace for your ...
3,When should I log to my personal entity agains...,You should log to your personal entity when yo...
4,Who can create a team? Who can add or delete p...,**Admin**: Team admins can add and remove othe...


Let's split into train/valid

In [43]:
df_train=df.sample(frac=0.9,random_state=200)
df_eval=df.drop(df_train.index)
len(df_train), len(df_eval)

(114, 13)

A neat trick to get better answers is instead of passing a very long initial message, passing a small conversation with some prefilled agent responses.

In [7]:
def create_messages(question: str, cls=ChatMessage):
    messages = [
        cls(
            role="user", 
            content=(
                "You are an expert about Weights & Biases the ML platform. "
                 "You will answer questions about the product, Answer the question directly, without repeating the instructions."
                 )
        ),
        cls(
            role="assistant", 
            content=(
                "Sure, I'd be happy to help with your question about Weights & Biases. "
                 "If you have a specific question about using Weights & Biases, such as how to track experiments, "
                 "visualize data, or manage artifacts, please feel free to ask!")
        ),
        cls(
            role="user", 
            content=f"Here is the question: {question}"
        )
    ]
    return messages

In [10]:
@weave.op()
async def wandb_expert(question:str, model:str) -> str:
    "Answer questions about wandb"
     
    messages = create_messages(question=question)

    answer = await call_mistral(model=model, messages=messages)
    return {"question": question, "answer": answer}

res = await wandb_expert(question=df.loc[0].question, model="mistral-medium-latest")
print(df.loc[0].question)
print(res["answer"])

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/264ede4c-e236-4805-a3ef-54d62c4c763c
What is the difference between team and organization?
In Weights & Biases (W&B), the difference between a team and an organization is primarily in terms of access control and collaboration.

A team is a group of users who are working together on a specific project or set of experiments. Teams can be created and managed by any W&B user, and team members can be added or removed as needed. Teams can also have different access levels for different members, such as read-only or admin access.

An organization, on the other hand, is a higher-level entity that can contain multiple teams. Organizations are typically used to manage larger groups of users, such as a company or research lab, and provide centralized management and administration of W&B resources. Organizations can also have different access levels for different members, and can have custom branding and billing options.

In summary, teams are use

## GT dataset
Let's create a dataset with mistral-medium-latest as our baseline

In [11]:
class MistralModel(weave.Model):
    model: str
    temperature: float = 0.7
    
    @weave.op
    def create_messages(self, question:str):
        return create_messages(question)

    @weave.op
    async def predict(self, question:str):
        messages = self.create_messages(question)
        return await call_mistral(model=self.model, messages=messages)

In [45]:
ds_train = weave.Dataset(name="ds_train", rows=df_train)
ds_eval = weave.Dataset(name="ds_eval", rows=df_eval)

let's publish them to Weave

In [47]:
weave.publish(ds_train)
weave.publish(ds_eval)

📦 Published to https://wandb.ai/capecape/mistral_webinar/weave/objects/ds_train/versions/ZFlKJFzLHbwN6w5bxi1pVRkkBiYZNF4zEqHrKUSDkYI
📦 Published to https://wandb.ai/capecape/mistral_webinar/weave/objects/ds_eval/versions/6nj1RQhTJNCezToyNmZNScj7MCHYKCjBoLMvuNHDeBE


ObjectRef(entity='capecape', project='mistral_webinar', name='ds_eval', digest='6nj1RQhTJNCezToyNmZNScj7MCHYKCjBoLMvuNHDeBE', extra=[])

Lets create a dataset with the medium model predictions

In [50]:
mistral_medium = MistralModel(model="mistral-medium-latest")

In [67]:
async def async_foreach(sequence, func, max_concurrent_tasks):
    "Handy parallelism async for looper"
    semaphore = asyncio.Semaphore(max_concurrent_tasks)
    async def process_item(item):
        async with semaphore:
            result = await func(item)
            return item, result

    tasks = [asyncio.create_task(process_item(item)) for item in sequence]

    for task in asyncio.as_completed(tasks):
        item, result = await task
        yield item, result

In [68]:
async def map(ds, func, max_concurrent_tasks = 7, col_name="mistral_medium"):
    new_dataset = []
    async for example, map_results in async_foreach(ds.rows, func, max_concurrent_tasks):
        example.update({col_name: map_results})
        new_dataset.append(example)
    return new_dataset

ds_eval_medium_rows = await map(ds_eval, mistral_medium.predict, col_name="mistral-medium")

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/a1d56ca5-4848-4b81-84e8-1ddaf5d8b3e0
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/4762498a-f2e8-4101-87bf-3033e7feb2b7
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/169e279e-cb74-4a5e-bf9e-465b605e7fce
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/b6bc1c9b-d71e-41c2-8562-441b2e665c6e
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/08a7a5f8-c85e-4a97-aaca-8d805b9ec49f
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/3ee07b19-9553-4f32-8a67-53a8fdee90b0
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/00aef0ec-76ef-47d1-ac18-e3b642437f9b
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/8f09d532-e6ef-4cb2-a1d7-d8b0f6f4d9ff


In [ ]:
ds_eval_medium = weave.Dataset(name="ds_eval_medium", description="Mistral medium predictions", rows=ds_eval_medium_rows)
weave.publish(ds_eval_medium)

📦 Published to https://wandb.ai/capecape/mistral_webinar/weave/objects/ds_eval_medium/versions/JX9ZhLO7D0TU9R0d8LSBiluUnVSByahEqtoy7skrqnQ


ObjectRef(entity='capecape', project='mistral_webinar', name='ds_eval_medium', digest='JX9ZhLO7D0TU9R0d8LSBiluUnVSByahEqtoy7skrqnQ', extra=[])

You can pull your data back easily using the API:

In [ ]:
ds_eval_medium = weave.ref('ds_eval_medium:v0').get()

In [ ]:
list(ds_eval_medium.rows)

[TraceDict({'question': 'How often are system metrics collected?', 'answer': 'By default, metrics are collected every 2 seconds and averaged over a 15-second period. If you need higher resolution metrics, email us a [contact@wandb.com](mailto:contact@wandb.com).', 'mistral-medium': 'In Weights & Biases, system metrics are collected by default every 2 seconds and then averaged over a 15-second period. However, if you need higher resolution metrics, you can contact Weights & Biases support at [contact@wandb.com](mailto:contact@wandb.com) to request customization of the metric collection interval.'}),
 TraceDict({'question': 'What is the difference between `.log()` and `.summary`?', 'answer': 'The summary is the value that shows in the table while the log will save all the values for plotting later.\n\nFor example, you might want to call `wandb.log` every time the accuracy changes. Usually, you can just use .log. `wandb.log()` will also update the summary value by default unless you have 

Let's add the results of Mistral 7B (non finetuned)

In [ ]:
mistral_7b = MistralModel(model="open-mistral-7b")
ds_eval_7b_rows = await map(ds_eval_medium, mistral_7b.predict, col_name="mistral_7b")
ds_eval_7b = weave.Dataset(name="ds_eval_medium_7b", description="Mistral 7b predictions", rows=ds_eval_7b_rows)
weave.publish(ds_eval_7b)

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/6caf9e2e-4677-4bec-adfb-fd676143211c
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/2171b067-d319-45e1-8ad6-02d83c8c0cc0
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/6394d286-b455-4d38-8136-a42623f8b9f4
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/bea011b4-1564-4b01-ab3f-2570cbfbbb04
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/b288e3ba-4357-4e0c-85b2-dd2c46f66ae5
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/11c07aad-438c-42ae-943b-4dc2e21ea4a1
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/4833ab50-9794-4559-85f1-88100153953c
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/f36b04a6-3b9c-4799-9bec-35851b39e493
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/3375bb57-5c55-4130-a187-dda4b6d301f0
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/a1b43519-24da-496d-834a-6d3820daec5c
🍩 https://wandb.ai/capecape/mistral_webinar/r/call/c8fe5ed4-699f-4136-bae8-2c8791ad9e96
🍩 https://wandb.ai/capecape/mist

ObjectRef(entity='capecape', project='mistral_webinar', name='ds_eval_medium_7b', digest='xdrfHXg1hhfxDiUdJsvLiAM15aE9r0tD7YIPMTXEzrM', extra=[])

## Evaluation
Let's use mistral large as a judge, let's compute a score as baseline comparing `7B` and `medium`.


In [64]:
class LLMJudge(weave.Model):
    model: str = "mistral-large-latest"
    
    @weave.op
    async def predict(self, question: str, mistral_7b: str, mistral_medium: str, answer: str) -> dict:
        messages = [
            ChatMessage(
                role="user",
                content=(
                "You are an expert about Weights & Biases the ML platform. "
                "You have to pick the best answer between two answers. "
                "Take into consideration the context of the question and the ground truth answer as a reference. \n"
                "Here is the question: {question}\n"
                "Here is the answer1: {answer_7b}\n"
                "Here is the answer2: {answer_medium}\n"
                "Ground truth answer: {answer}\n"
                "Return the name of the best_answer and the reason in short JSON object.").format(
                    question=question, 
                    answer_7b=answer_7b, 
                    answer_medium=answer_medium,
                    answer=answer)
            )
        ]
        payload = await call_mistral(model=self.model, messages=messages, response_format={"type": "json_object"})
        return json.loads(payload)

In [66]:
ds_eval_7b.rows[0].keys()

dict_keys(['question', 'answer', 'mistral-medium', 'mistral-7b'])

In [ ]:
judge = LLMJudge()
judge.predict(df.loc[0].question, res["answer"], res["answer"])



In [105]:
llm_judge(df.loc[0].question, res["answer"], res["answer"])

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/6cf3932a-3c16-4e68-bff3-efc92a0d45e3


{'best_answer': 'answer1',
 'reason': 'Both answers are identical, but answer1 was provided first.'}

In [109]:
@weave.op
def evaluate_answer(question: str, model_output: str) -> dict:
    "Evaluate the answer"
    judgement = llm_judge(question, model_output, answer_medium, answer)
    return {"win": judgement["best_answer"] == "answer1"}

Let's define a weave.evaluation

In [110]:
df

,question,answer
0,What is the difference between team and organi...,A team is a collaborative workspace for a grou...
1,What is the difference between team and entity...,A team is a collaborative workspace for a grou...
2,What is a team and where can I find more infor...,Use W&B Teams as a central workspace for your ...
3,When should I log to my personal entity agains...,You should log to your personal entity when yo...
4,Who can create a team? Who can add or delete p...,**Admin**: Team admins can add and remove othe...
...,...,...
122,How do I find an artifact from the best run in...,You can use the following code to retrieve the...
123,How do I save code in an artifact?‌,Use `save_code=True` in `wandb.init` to save t...
124,Using artifacts with multiple architectures an...,There are many ways in which you can think of ...
125,How can I fetch these Version IDs and ETags in...,If you've logged an artifact reference with W&...


In [111]:
evaluation = weave.Evaluation(dataset=df.iloc[0:10].to_dict(orient="records"), scorers=[evaluate_answer])

In [112]:
await evaluation.evaluate(mistral_7b)

Evaluated 1 of 10 examples

Evaluated 2 of 10 examples

Evaluated 3 of 10 examples

Evaluated 4 of 10 examples

Evaluated 5 of 10 examples

...

Evaluated 9 of 10 examples

Evaluated 10 of 10 examples

Evaluation summary
{
    'evaluate_answer': {'win': {'true_count': 10, 'true_fraction': 1.0}},
    'model_latency': {'mean': 5.810741329193116}
}

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/749dca23-7481-42ea-9cb0-d1c893719b4e


{'evaluate_answer': {'win': {'true_count': 10, 'true_fraction': 1.0}},
 'model_latency': {'mean': 5.810741329193116}}

This is pretty descent for both 😍. Let's see if fine-tuning improves this.

In [9]:
def format_messages(row):
    "Format on the expected MistralAI fine-tuning dataset"
    question = row['question']
    answer = row['answer']
    messages = create_messages(question, cls=dict)
    # we need to append the answer for training 👇
    messages = {"messages":messages + [dict(role="assistant", content=answer)]}
    return messages

In [10]:
msgs = format_messages(df.loc[0])
msgs

{'messages': [{'role': 'user',
   'content': 'You are an expert about Weights & Biases the ML platform. You will answer questions about the product, Answer the question directly, without repeating the instructions.'},
  {'role': 'assistant',
   'content': "Sure, I'd be happy to help with your question about Weights & Biases. If you have a specific question about using Weights & Biases, such as how to track experiments, visualize data, or manage artifacts, please feel free to ask!"},
  {'role': 'user',
   'content': 'Here is the question: What is the difference between team and organization?'},
  {'role': 'assistant',
   'content': 'A team is a collaborative workspace for a group of users working on the same projects, while an organization is a higher-level entity that may consist of multiple teams and is often related to billing and account management.'}]}

In [79]:
df = df.apply(format_messages, axis=1)
df.head()

0    {'messages': [{'role': 'user', 'content': 'You...
1    {'messages': [{'role': 'user', 'content': 'You...
2    {'messages': [{'role': 'user', 'content': 'You...
3    {'messages': [{'role': 'user', 'content': 'You...
4    {'messages': [{'role': 'user', 'content': 'You...
dtype: object

In [78]:
df_train.to_json("train.jsonl", orient="records", lines=True)
df_eval.to_json("eval.jsonl", orient="records", lines=True)

## Upload dataset

In [13]:
import os
from mistralai.client import MistralClient

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("train.jsonl", "rb") as f:
    ds_train = client.files.create(file=("train.jsonl", f))
with open("eval.jsonl", "rb") as f:
    ds_eval = client.files.create(file=("eval.jsonl", f))


In [14]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [15]:
pprint(ds_train)

{
    "id": "d40cc185-6f0d-4754-bc05-5db7f6e3723a",
    "object": "file",
    "bytes": 147176,
    "created_at": 1719343148,
    "filename": "train.jsonl",
    "purpose": "fine-tune"
}


In [16]:
pprint(ds_eval)

{
    "id": "2a5b6582-3e90-4af6-800e-e9cf2d904bda",
    "object": "file",
    "bytes": 15339,
    "created_at": 1719343148,
    "filename": "eval.jsonl",
    "purpose": "fine-tune"
}


## Create a fine-tuning job

In [20]:
from mistralai.models.jobs import TrainingParameters, WandbIntegrationIn

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[ds_train.id],
    validation_files=[ds_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=25,
        learning_rate=0.0001,
        ),
    integrations=[
        WandbIntegrationIn(
            project="mistral_webinar",
            run_name="finetune_wandb",
            api_key=os.environ.get("WANDB_API_KEY"),
        ).dict()
    ],
)

In [21]:
pprint(created_jobs)

{
    "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
    "hyperparameters": {
        "training_steps": 25,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1719343209,
    "modified_at": 1719343209,
    "training_files": [
        "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
    ],
    "validation_files": [
        "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ]
}


In [22]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
    "hyperparameters": {
        "training_steps": 25,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "open-mistral-7b",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1719343209,
    "modified_at": 1719343210,
    "training_files": [
        "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
    ],
    "validation_files": [
        "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "RUNNING"
            },
            "created_at": 1719343210
        },
        {
            "name": "status-updated",
            "data": {
                "status": "QUEUED"
            },
            "

In [23]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
            "hyperparameters": {
                "training_steps": 25,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:open-mistral-7b:0362203c:20240625:a063d186",
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1719343209,
            "modified_at": 1719343355,
            "training_files": [
                "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
            ],
            "validation_files": [
                "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
            ],
            "object": "job",
            "integrations": [
                {
                    "type": "wandb",
                    "project": "mistral_webinar",
                    "name": null,
                    "run_name": "finetune_wandb"
                }
            ]
        },
        {
            "id": "5e4e

In [24]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "a063d186-3eab-4b65-8fa3-bb081083e006",
    "hyperparameters": {
        "training_steps": 25,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:open-mistral-7b:0362203c:20240625:a063d186",
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1719343209,
    "modified_at": 1719343355,
    "training_files": [
        "d40cc185-6f0d-4754-bc05-5db7f6e3723a"
    ],
    "validation_files": [
        "2a5b6582-3e90-4af6-800e-e9cf2d904bda"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "mistral_webinar",
            "name": null,
            "run_name": "finetune_wandb"
        }
    ],
    "events": [
        {
            "name": "status-updated",
            "data": {
                "status": "SUCCESS"
            },
            "created_at": 1719343355
        },
        {
            "name": "status-updated",
            "data": {
                "sta

## Use a fine-tuned model

In [25]:
df_eval.iloc[1]

{'messages': [{'role': 'user',
   'content': 'You are an expert about Weights & Biases the ML platform. You will answer questions about the product, Answer the question directly, without repeating the instructions.'},
  {'role': 'assistant',
   'content': "Sure, I'd be happy to help with your question about Weights & Biases. If you have a specific question about using Weights & Biases, such as how to track experiments, visualize data, or manage artifacts, please feel free to ask!"},
  {'role': 'user',
   'content': 'Here is the question: What is the difference between `.log()` and `.summary`?'},
  {'role': 'assistant',
   'content': 'The summary is the value that shows in the table while the log will save all the values for plotting later.\n\nFor example, you might want to call `wandb.log` every time the accuracy changes. Usually, you can just use .log. `wandb.log()` will also update the summary value by default unless you have set the summary manually for that metric\n\nThe scatterplo

In [26]:
wandb_expert(question="What is the difference between `.log()` and `.summary`?", 
             model=retrieved_jobs.fine_tuned_model)

🍩 https://wandb.ai/capecape/mistral_webinar/r/call/eaccb136-f7d7-4191-a97f-2aaf8ebcf90a


{'question': 'What is the difference between `.log()` and `.summary`?',
 'answer': 'Use `.log()` to record scalar values that are not gradients, like loss or accuracy. Use `.summary()` to record scalars that are gradients (also called "Vanilla scalars" in the W&B App). The difference between logging scalars with `.log()` and `.summary()` is that `.summary()` automatically scales the scalars by the magnitude of the largest gradient seen so far for that scalar. This scaling makes it easier to compare different scalars with different units, such as loss (which is typically in the range [0, 1]) and learning rate (which is typically in the range [0, 1e-4]). For example, if you log both loss and learning rate with `.log()`, you\'ll have to manually scale the learning rate to compare it to the loss. If you log both with `.summary()`, W&B will automatically scale the learning rate for you.'}